<a href="https://colab.research.google.com/github/dudaholandah/NLP/blob/main/Projeto/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r '/content/drive/MyDrive/Datasets/cuisine-classification-ingredients' 'cuisine-classification-ingredients'

In [ ]:
!cp -r '/content/drive/MyDrive/Datasets/indian-food' 'indian-food'

# Imports

In [ ]:
!pip install -U sentence-transformers

In [5]:
!pip install keras
!pip install tensorflow

In [6]:
#TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
#WORD2VEC
from gensim.models import Word2Vec
import nltk
#BERT
import tensorflow as tf
from sentence_transformers import SentenceTransformer
#Neural Network
from keras.models import Sequential
from keras.layers import Dense,Conv1D,Embedding,GlobalMaxPooling1D,MaxPooling1D,Activation,Flatten,SimpleRNN,Dropout
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.optimizers import SGD
#Data
import json
import os
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

In [ ]:
nltk.download('punkt')

# Data

Cuisine Classification Data

In [8]:
train_path = 'cuisine-classification-ingredients/train.json'
# test_path = 'cuisine-classification-ingredients/test.json'

with open(train_path, 'r') as f:
  train_json = json.load(f)

# with open(test_path, 'r') as f:
#   test_json = json.load(f)

In [9]:
df_train_cuisine = pd.DataFrame(train_json, columns=['id', 'cuisine', 'ingredients'])
# df_test_cuisine = pd.DataFrame(test_json, columns=['id', 'ingredients'])

In [29]:
df_indian['course'].value_counts()

main course    129
dessert         85
snack           39
starter          2
Name: course, dtype: int64

In [11]:
df_train_cuisine = df_train_cuisine.drop(['id'], axis=1)
# df_test_cuisine = df_test_cuisine.drop(['id'], axis=1)

Indian Food Data

In [12]:
path = 'indian-food/indian_food.csv'
df_indian = pd.read_csv(path)

In [22]:
df_indian

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,maida flour yogurt oil sugar,vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,gram flour ghee sugar,vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,carrots milk sugar ghee cashews raisins,vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,flour ghee kewra milk clarified butter sugar a...,vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,milk powder plain flour baking powder ghee mil...,vegetarian,15,40,sweet,dessert,West Bengal,East
...,...,...,...,...,...,...,...,...,...
250,Til Pitha,glutinous rice black sesame seeds gur,vegetarian,5,30,sweet,dessert,Assam,North East
251,Bebinca,coconut milk egg yolks clarified butter all pu...,vegetarian,20,60,sweet,dessert,Goa,West
252,Shufta,cottage cheese dry dates dried rose petals pis...,vegetarian,-1,-1,sweet,dessert,Jammu & Kashmir,North
253,Mawa Bati,milk powder dry fruits arrowroot powder all pu...,vegetarian,20,45,sweet,dessert,Madhya Pradesh,Central


## Pre-Processing

In [14]:
def pre_processing(text):
  new = ""
  for word in text.split(','):
    word = re.sub(r'[^\w\s]', '', word.lower())
    word = re.sub(r'[0-9]+', '', word)
    word = re.sub(r'\boz\b', '', word)
    word = re.sub(r'\ba taste of\b', '', word)
    new = new + word.strip() + " "

  return new[:-1]

Cuisine Classification Data

In [15]:
df_train_cuisine['ingredients'] = [','.join(x).strip() for x in df_train_cuisine['ingredients']]
# df_test_cuisine['ingredients'] = [','.join(x).strip() for x in df_test_cuisine['ingredients']]

In [16]:
df_train_cuisine['ingredients'] = [pre_processing(x) for x in df_train_cuisine['ingredients']]
# df_test_cuisine['ingredients'] = [pre_processing(x) for x in df_test_cuisine['ingredients']]

In [17]:
X = df_train_cuisine['ingredients']
y = df_train_cuisine['cuisine']

# dados de teste = 20% do treinamento
X_train_cuisine, X_test_cuisine, y_train_cuisine, y_test_cuisine = train_test_split(X, y, test_size=0.2)

# dados de validacao = 50% do teste
X_test_cuisine, X_valid_cuisine, y_test_cuisine, y_valid_cuisine = train_test_split(X, y, test_size=0.5)

In [18]:
y_train_cuisine = pd.Categorical(y_train_cuisine)
y_train_int = y_train_cuisine.codes

y_valid_cuisine = pd.Categorical(y_valid_cuisine)
y_valid_int = y_valid_cuisine.codes

y_test_cuisine = pd.Categorical(y_test_cuisine)
y_test_int = y_test_cuisine.codes

y_train_cuisine = to_categorical(y_train_int)
y_valid_cuisine = to_categorical(y_valid_int)
y_test_cuisine = to_categorical(y_test_int)

Indian Food Data

In [19]:
df_indian['ingredients'] = [pre_processing(x) for x in df_indian['ingredients']]

In [20]:
X = df_indian['ingredients']
y = df_indian['diet']

# dados de teste = 20% do treinamento
X_train_indian, X_test_indian, y_train_indian, y_test_indian = train_test_split(X, y, test_size=0.2)

# dados de validacao = 50% do teste
X_test_indian, X_valid_indian, y_test_indian, y_valid_indian = train_test_split(X, y, test_size=0.5)

In [21]:
y_train_indian = pd.Categorical(y_train_indian)
y_train_int = y_train_indian.codes

y_valid_indian = pd.Categorical(y_valid_indian)
y_valid_int = y_valid_indian.codes

y_test_indian = pd.Categorical(y_test_indian)
y_test_int = y_test_indian.codes

y_train_indian = to_categorical(y_train_int)
y_valid_indian = to_categorical(y_valid_int)
y_test_indian = to_categorical(y_test_int)

# Different Embeddings

In [146]:
vocabulary_cuisine = {}
i = 0

for ingredients in df_train_cuisine['ingredients']:
  for word in ingredients.split():
    if word not in vocabulary_cuisine:
      vocabulary_cuisine[word] = i
      i += 1

In [197]:
vocabulary_cuisine

{'romaine': 0,
 'lettuce': 1,
 'black': 2,
 'olives': 3,
 'grape': 4,
 'tomatoes': 5,
 'garlic': 6,
 'pepper': 7,
 'purple': 8,
 'onion': 9,
 'seasoning': 10,
 'garbanzo': 11,
 'beans': 12,
 'feta': 13,
 'cheese': 14,
 'crumbles': 15,
 'plain': 16,
 'flour': 17,
 'ground': 18,
 'salt': 19,
 'thyme': 20,
 'eggs': 21,
 'green': 22,
 'yellow': 23,
 'corn': 24,
 'meal': 25,
 'milk': 26,
 'vegetable': 27,
 'oil': 28,
 'mayonaise': 29,
 'cooking': 30,
 'chilies': 31,
 'grilled': 32,
 'chicken': 33,
 'breasts': 34,
 'powder': 35,
 'soy': 36,
 'sauce': 37,
 'butter': 38,
 'livers': 39,
 'water': 40,
 'wheat': 41,
 'shallots': 42,
 'cornflour': 43,
 'cayenne': 44,
 'onions': 45,
 'paste': 46,
 'lemon': 47,
 'juice': 48,
 'chili': 49,
 'passata': 50,
 'cumin': 51,
 'boneless': 52,
 'skinless': 53,
 'thigh': 54,
 'garam': 55,
 'masala': 56,
 'double': 57,
 'cream': 58,
 'natural': 59,
 'yogurt': 60,
 'bay': 61,
 'leaf': 62,
 'sugar': 63,
 'fresh': 64,
 'ginger': 65,
 'root': 66,
 'cinnamon': 67,


In [159]:
vocabulary_indian = {}
i = 0

for ingredients in df_indian['ingredients']:
  for word in ingredients.split():
    if word not in vocabulary_indian:
      vocabulary_indian[word] = i
      i += 1

## TFIDF

Cuisine Classification Data

In [23]:
X_train_cuisine

19576    shortening yellow corn meal buttermilk butter ...
23974    green olives extravirgin olive oil baguette ga...
13467    red pepper flakes cane sugar sake awase miso v...
18680    kosher salt fresh thyme red snapper olive oil ...
10623    baking soda allpurpose flour sour cream powder...
                               ...                        
25460                 radishes olives baguette fresh thyme
9084     vegetable oil ground cumin picante sauce bonel...
12046    fresh parmesan cheese stewed tomatoes cooking ...
14962    water peanut oil chinese rice wine garlic gree...
9155     large eggs salt sugar butter baking powder all...
Name: ingredients, Length: 31819, dtype: object

In [149]:
tfidf_vectorizer = TfidfVectorizer(vocabulary=vocabulary_cuisine)

In [150]:
# Training Data
tfidf_train_cuisine = tfidf_vectorizer.fit_transform(X_train_cuisine)
tfidf_train_cuisine = tfidf_train_cuisine.toarray()
df_tfidf_train_cuisine = pd.DataFrame(tfidf_train_cuisine, columns=tfidf_vectorizer.get_feature_names_out())

In [155]:
df_tfidf_train_cuisine.shape

(31819, 3064)

In [151]:
# Validation Data
tfidf_valid_cuisine = tfidf_vectorizer.fit_transform(X_valid_cuisine)
tfidf_valid_cuisine = tfidf_valid_cuisine.toarray()
df_tfidf_valid_cuisine = pd.DataFrame(tfidf_valid_cuisine, columns=tfidf_vectorizer.get_feature_names_out())

In [156]:
df_tfidf_valid_cuisine.shape

(19887, 3064)

Indian Food Data

In [161]:
tfidf_vectorizer = TfidfVectorizer(vocabulary=vocabulary_indian)

In [162]:
# Training Data
tfidf_train_indian = tfidf_vectorizer.fit_transform(X_train_indian)
tfidf_train_indian = tfidf_train_indian.toarray()
df_tfidf_train_indian = pd.DataFrame(tfidf_train_indian, columns=tfidf_vectorizer.get_feature_names_out())

In [164]:
df_tfidf_train_indian.shape

(204, 337)

In [163]:
# Validation Data
tfidf_valid_indian = tfidf_vectorizer.fit_transform(X_valid_indian)
tfidf_valid_indian = tfidf_valid_indian.toarray()
df_tfidf_valid_indian = pd.DataFrame(tfidf_valid_indian, columns=tfidf_vectorizer.get_feature_names_out())

In [165]:
df_tfidf_valid_indian.shape

(128, 337)

## Word2Vec

In [87]:
def get_embedding_w2v(model, doc_tokens):
    embeddings = []
    for tok in doc_tokens:
        if tok in model.wv.vocab:
                embeddings.append(model.wv.word_vec(tok))
        else:
            embeddings.append(np.random.rand(300))
    return np.mean(embeddings, axis=0)

Cuisine Classification Data

In [190]:
ingredients = '. '.join(X_train_cuisine)
sentences = nltk.sent_tokenize(ingredients)
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
model_word2vec = Word2Vec(sentences, size=300, min_count=1, window=5, sg=1,workers=4)

In [191]:
X_train_word2vec_cuisine = X_train_cuisine.apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))

In [192]:
ingredients = '. '.join(X_valid_cuisine)
sentences = nltk.sent_tokenize(ingredients)
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
model_word2vec = Word2Vec(sentences, size=300, min_count=1, window=5, sg=1,workers=4)

In [193]:
X_valid_word2vec_cuisine = X_valid_cuisine.apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))

Indian Food Data

In [99]:
ingredients = '. '.join(X_train_indian)
sentences = nltk.sent_tokenize(ingredients)
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
model_word2vec = Word2Vec(sentences, size=300, min_count=1, window=5, sg=1,workers=1)

In [101]:
X_train_word2vec_indian = X_train_indian.apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))

In [110]:
ingredients = '. '.join(X_valid_indian)
sentences = nltk.sent_tokenize(ingredients)
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
model_word2vec = Word2Vec(sentences, size=300, min_count=1, window=5, sg=1,workers=1)

In [111]:
X_valid_word2vec_indian = X_valid_indian.apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))

## BERT

In [42]:
bert_model_sentence = SentenceTransformer('bert-base-nli-mean-tokens')

In [45]:
sentences_bert = bert_model_sentence.encode(X_train)

In [48]:
len(train_tfidf[0])

3002

In [47]:
len(sentences_word2vec[0])

300

In [46]:
len(sentences_bert[0])

768

# Different Classificators

In [166]:
vocab_size_cuisine = len(vocabulary_cuisine)
vocab_size_indian = len(vocabulary_indian)

## CNN

In [54]:
def build_model_CNN(vocab_size, n, max_length, output):
  model = Sequential()
  model.add(Embedding(vocab_size, output_dim = n, input_length=max_length))
  model.add(Conv1D(filters=64, kernel_size=11, strides=1))
  model.add(Activation('relu'))
  model.add(GlobalMaxPooling1D())
  model.add(Dense(128))
  model.add(Activation('relu'))
  model.add(Dense(64))
  model.add(Activation('relu'))
  model.add(Dense(output, activation='softmax'))
  return model

### TFIDF

Cuisine Classification Data

In [168]:
model_cuisine = build_model_CNN(vocab_size_cuisine, 32, vocab_size_cuisine, 20)

In [ ]:
sgd = SGD(learning_rate=0.001)
model_cuisine.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=['accuracy'])
history_fine = model_cuisine.fit(df_tfidf_train_cuisine,y_train_cuisine,validation_data=(df_tfidf_valid_cuisine,y_valid_cuisine),epochs=20)

Indian Food Data

In [172]:
model_indian = build_model_CNN(vocab_size_indian, 32, vocab_size_indian, 2)

In [ ]:
sgd = SGD(learning_rate=0.001)
model_indian.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=['accuracy'])
history_fine = model_indian.fit(df_tfidf_train_indian,y_train_indian,validation_data=(df_tfidf_valid_indian,y_valid_indian),epochs=20)

### Word2vec

Cuisine Classification Data

In [103]:
model_cuisine = build_model_CNN(vocab_size_cuisine, 32, 300, 20)

In [105]:
X_train_word2vec_cuisine = np.array([np.array(val) for val in X_train_word2vec_cuisine])
X_valid_word2vec_cuisine = np.array([np.array(val) for val in X_valid_word2vec_cuisine])

In [106]:
sgd = SGD(learning_rate=0.001)
model_cuisine.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=['accuracy'])
history_fine = model_cuisine.fit(X_train_word2vec_cuisine,y_train_cuisine,validation_data=(X_valid_word2vec_cuisine,y_valid_cuisine) , epochs=20)

Epoch 1/20
1119/1119 [==============================] - 5s 4ms/step - loss: 2.9427 - accuracy: 0.1860 - val_loss: 2.8962 - val_accuracy: 0.2001
Epoch 2/20
1119/1119 [==============================] - 4s 4ms/step - loss: 2.8466 - accuracy: 0.1967 - val_loss: 2.7986 - val_accuracy: 0.2001
Epoch 3/20
1119/1119 [==============================] - 5s 4ms/step - loss: 2.7459 - accuracy: 0.1967 - val_loss: 2.7014 - val_accuracy: 0.2001
Epoch 4/20
1119/1119 [==============================] - 5s 4ms/step - loss: 2.6673 - accuracy: 0.1967 - val_loss: 2.6495 - val_accuracy: 0.2001
Epoch 5/20
1119/1119 [==============================] - 4s 4ms/step - loss: 2.6355 - accuracy: 0.1967 - val_loss: 2.6310 - val_accuracy: 0.2001
Epoch 6/20
1119/1119 [==============================] - 5s 4ms/step - loss: 2.6227 - accuracy: 0.1967 - val_loss: 2.6214 - val_accuracy: 0.2001
Epoch 7/20
1119/1119 [==============================] - 5s 4ms/step - loss: 2.6159 - accuracy: 0.1967 - val_loss: 2.6162 - val_accuracy:

Indian Food Data

In [176]:
model_indian = build_model_CNN(vocab_size_indian, 32, 300, 2)

In [177]:
X_train_word2vec_indian = np.array([np.array(val) for val in X_train_word2vec_indian])
X_valid_word2vec_indian = np.array([np.array(val) for val in X_valid_word2vec_indian])

In [178]:
sgd = SGD(learning_rate=0.001)
model_indian.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=['accuracy'])
history_fine = model_indian.fit(X_train_word2vec_indian,y_train_indian,validation_data=(X_valid_word2vec_indian,y_valid_indian) , epochs=20)

Epoch 1/20
7/7 [==============================] - 1s 40ms/step - loss: 0.6936 - accuracy: 0.3676 - val_loss: 0.6899 - val_accuracy: 0.9062
Epoch 2/20
7/7 [==============================] - 0s 19ms/step - loss: 0.6878 - accuracy: 0.8873 - val_loss: 0.6838 - val_accuracy: 0.9062
Epoch 3/20
7/7 [==============================] - 0s 13ms/step - loss: 0.6824 - accuracy: 0.8873 - val_loss: 0.6788 - val_accuracy: 0.9062
Epoch 4/20
7/7 [==============================] - 0s 14ms/step - loss: 0.6777 - accuracy: 0.8873 - val_loss: 0.6741 - val_accuracy: 0.9062
Epoch 5/20
7/7 [==============================] - 0s 12ms/step - loss: 0.6732 - accuracy: 0.8873 - val_loss: 0.6693 - val_accuracy: 0.9062
Epoch 6/20
7/7 [==============================] - 0s 13ms/step - loss: 0.6687 - accuracy: 0.8873 - val_loss: 0.6647 - val_accuracy: 0.9062
Epoch 7/20
7/7 [==============================] - 0s 11ms/step - loss: 0.6644 - accuracy: 0.8873 - val_loss: 0.6601 - val_accuracy: 0.9062
Epoch 8/20
7/7 [===========

## RNN

In [183]:
def build_model_RNN(vocab_size, n, max_length, output):
  model = Sequential()
  model.add(Embedding(vocab_size, output_dim = n, input_length=max_length))
  model.add(SimpleRNN(128,return_sequences=True,activation='relu'))
  model.add(Dropout(0.5))
  model.add(SimpleRNN(64,activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(output, activation="softmax"))
  return model

### Word2vec

Cuisine Classification Data

In [187]:
model_cuisine = build_model_RNN(vocab_size_cuisine, 32, 300, 20)

In [194]:
X_train_word2vec_cuisine = np.array([np.array(val) for val in X_train_word2vec_cuisine])
X_valid_word2vec_cuisine = np.array([np.array(val) for val in X_valid_word2vec_cuisine])

In [ ]:
sgd = SGD(learning_rate=0.001)
model_cuisine.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=['accuracy'])
history_fine = model_cuisine.fit(X_train_word2vec_cuisine,y_train_cuisine,validation_data=(X_valid_word2vec_cuisine,y_valid_cuisine) , epochs=20)

Indian Food Data

In [184]:
model_indian = build_model_RNN(vocab_size_indian, 32, 300, 2)

In [185]:
X_train_word2vec_indian = np.array([np.array(val) for val in X_train_word2vec_indian])
X_valid_word2vec_indian = np.array([np.array(val) for val in X_valid_word2vec_indian])

In [186]:
sgd = SGD(learning_rate=0.001)
model_indian.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=['accuracy'])
history_fine = model_indian.fit(X_train_word2vec_indian,y_train_indian,validation_data=(X_valid_word2vec_indian,y_valid_indian) , epochs=20)

Epoch 1/20
7/7 [==============================] - 6s 682ms/step - loss: 0.6884 - accuracy: 0.5588 - val_loss: 0.6778 - val_accuracy: 0.9062
Epoch 2/20
7/7 [==============================] - 4s 613ms/step - loss: 0.6740 - accuracy: 0.6569 - val_loss: 0.6620 - val_accuracy: 0.9062
Epoch 3/20
7/7 [==============================] - 4s 631ms/step - loss: 0.6675 - accuracy: 0.6765 - val_loss: 0.6479 - val_accuracy: 0.9062
Epoch 4/20
7/7 [==============================] - 4s 629ms/step - loss: 0.6518 - accuracy: 0.7647 - val_loss: 0.6333 - val_accuracy: 0.9062
Epoch 5/20
7/7 [==============================] - 4s 573ms/step - loss: 0.6404 - accuracy: 0.8088 - val_loss: 0.6207 - val_accuracy: 0.9062
Epoch 6/20
7/7 [==============================] - 4s 600ms/step - loss: 0.6421 - accuracy: 0.7990 - val_loss: 0.6086 - val_accuracy: 0.9062
Epoch 7/20
7/7 [==============================] - 4s 585ms/step - loss: 0.6210 - accuracy: 0.8627 - val_loss: 0.5976 - val_accuracy: 0.9062
Epoch 8/20
7/7 [====